# Introduction

The Future Sales competition is the final assesment in the 'How to win a Data Science' course in the Advanced Machine Learning specialisation from HSE University, Moscow. The aim is to predict the monthly sales of items in specific shops, given historical data. The sale counts are clipped between 0 and 20.

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

필요한 라이브러리를 불러오고 데이터 파일의 경로를 출력

# Load Data

In [ ]:
# load data
items=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
shops=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
cats=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
train=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

CSV 파일로부터 데이터를 로드 -> 데이터 프레임 생성됨

# 1. Data Cleaning

We'll remove outliers, clean up some of the raw data and add some new variables to it.

# Remove outliers

이상치(다른 데이터 포인트와 비교했을 때 현저히 다른 값) 제거 

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
flierprops = dict(marker='o', markerfacecolor='purple', markersize=6,
                  linestyle='none', markeredgecolor='black')
sns.boxplot(x=train.item_cnt_day, flierprops=flierprops)

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price, flierprops=flierprops)

1) 첫 번째 박스 플롯 'item_cnt_day'
그림의 크기를 가로 10인치, 세로 4인치로 설정    
x축의 범위를 -100에서 30000까지 설정(박스플롯에서 데이터 분포를 더 잘 보기 위해)
flierprops : 이상치(outliers)의 스타일 지정
             원형 마커, 마커 내부 색상은 보라색, 마커 크기는 6, 선 스타일 없애고, 마커 테두리 색상 검정
item_cnt_day열의 값을 사용하여 박스 플롯 생성

2) 두 번째 박스 플롯 'item_price'
그림의 크기를 가로 10인치 세로 4인치로 설정
x축의 범위를 item_price의 최소값에서 최대값의 1.1배까지 설정
item_price 열의 값을 사용하여 박스 플롯 생성

아이템의 가격이 300,000보다 작은 데이터 && 아이템 판매 수가 1000보다 작은 데이터만 선택 ( 이상치 제거 )

In [ ]:
train = train[(train.item_price < 300000 )& (train.item_cnt_day < 1000)]

아이템 가격이 0보다 큰 행만 필터링하여 train 데이터프레임에 저장 (가격이 0 이하 제거) 필터링 후 인덱스 재설정 drop=True 옵션을 사용하여 기존 인덱스 버림(필터링 후에도 인덱스가 연속적으로 유지되도록)
아이템 판매 수가 1미만인 경우 0으로 설정

In [ ]:
train = train[train.item_price > 0].reset_index(drop = True)
train.loc[train.item_cnt_day < 1, "item_cnt_day"] = 0

# Cleaning Shop Data

여러 상점이 서로 중복된 것처럼 보인다. 이는 상점이 다시 개업했거나 같은 거리나 쇼핑 센터 내에서 위치를 이동했기 때문일 수 있다

데이터셋에서 상점 ID를 업데이트하여 중복된 상점들을 하나로 통합하는 작업 수행

In [ ]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

train 데이터프레임에서 shop_id가 0인 행 찾아서 shop_id를 57로 변경
test 데이터프레임도 마찬가지
나머지도 같은 방식

일부 상점 이름 정리하고 city, category 정보를 shops 데이터프레임에 추가

In [ ]:
shops.loc[ shops.shop_name == 'Сергиев Посад ТЦ "7Я"',"shop_name" ] = 'СергиевПосад ТЦ "7Я"'
shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] )
shops["category"] = shops.shop_name.str.split(" ").map( lambda x: x[1] )
shops.loc[shops.city == "!Якутск", "city"] = "Якутск"

shop 데이터프레임에서 shop_name이 'Сергиев Посад ТЦ "7Я"'인 행 찾아서 shop_name 을 'СергиевПосад ТЦ "7Я"'로 수정 (이름 붙여쓰는 작업, 상점 이름의 일관성 유지위해)
shop_name을 공백' '으로 분리하여 첫 번째 단어를 city 열에 저장
shop_name을 공백' '으로 분리하여 두 번째 단어를 category 열에 저장
city 열에서 도시 이름이 "!Якутск"인 행 찾아서 "Якутск"으로 변경

특정 카테고리에 속하는 상점이 5개 이상인 경우에만 해당 카테고리 유지, 그렇지 않은 경우 'other'로 그룹화 -> 카테고리 정보 간소화

In [ ]:
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5:
        category.append(cat)
shops.category = shops.category.apply( lambda x: x if (x in category) else "other" )

빈 category 리스트 생성
shops 데이터프레임에서 고유한 카테고리를 추출하고, 각 카테고리를 순회
각 카테고리에 대해 해당 카테고리에 속하는 상점의 개수가 5개 이상인 경우
해당 카테고리를 category 리스트에 추가
'shops' 데이터프레임의 category 열을 순회해서 각 카테고리가 category 리스트에 있으면 유지 , 그렇지 않으면 other로 대체

In [ ]:
from sklearn.preprocessing import LabelEncoder
shops["shop_category"] = LabelEncoder().fit_transform( shops.category )
shops["shop_city"] = LabelEncoder().fit_transform( shops.city )
shops = shops[["shop_id", "shop_category", "shop_city"]]

LabelEncoder() 객체를 생성하고 fit_transform 메서드 사용해서 shops.category 열의 문자열 값을 숫자형으로 변환 -> shop_category라는 새로운 열에 저장
shop.city도 마찬가지
shop_id, shop_category, chop_city 열만 선택해서 'shops' 데이터프레임 재구성

# Cleaning Item Category Data

In [ ]:
cats["type_code"] = cats.item_category_name.apply( lambda x: x.split(" ")[0] ).astype(str)
cats.loc[ (cats.type_code == "Игровые")| (cats.type_code == "Аксессуары"), "category" ] = "Игры"

cats.item_category_name 열의 값을 공백' '으로 분리하여 첫 번째 단어를 추출 -> astype(str) 사용해서 결과를 문자열로 변환 -> type_code 열에 저장
loc를 사용하여 type_code 열의 값이 "Игровые" 또는 "Аксессуары"인 행을 선택해서 'category' 열 값을 "Игры"로 설정

In [ ]:
category = []
for cat in cats.type_code.unique():
    if len(cats[cats.type_code == cat]) >= 5: 
        category.append( cat )
cats.type_code = cats.type_code.apply(lambda x: x if (x in category) else "etc")

아까 shops 데이터프레임과 마찬가지로 type_code열을 간소화
(5개 이상 속하면 type_code 유지, 아니면 etc로 그룹화)

In [ ]:
cats.type_code = LabelEncoder().fit_transform(cats.type_code)
cats["split"] = cats.item_category_name.apply(lambda x: x.split("-"))
cats["subtype"] = cats.split.apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats["subtype_code"] = LabelEncoder().fit_transform( cats["subtype"] )
cats = cats[["item_category_id", "subtype_code", "type_code"]]

cats.type_code열 레이블 인코딩해서 문자열 값을 숫자형 값으로 변환
cats_category_name을 '-'으로 분리해서 split열에 저장
split 열의 값이 리스트로 되어 있으므로, 리스트의 두 번째 요소를 선택해서 'subtype'열에 저장(리스트 길이가 1이하면 리스트의 첫 번째 요소 선택)
strip 메서드를 사용해서 문자열의 앞뒤 공백 제거
subtype 열의 문자열 값을 레이블 인코딩해서 숫자형 값으로 변환하여 subtype_code열에 저장
필요한 열만 선택해서 데이터프레임 재구성

# Cleaning Item Data

In [ ]:
import re
def name_correction(x):
    x = x.lower() # 모든 문자를 소문자로 변환
    x = x.partition('[')[0] # 대괄호로 구분하여 첫 번째 부분만 남김
    x = x.partition('(')[0] # 소괄호로 구분하여 첫 번째 부분만 남김
    x = re.sub('[^A-Za-z0-9А-Яа-я]+', ' ', x) # 특수 문자를 제거하고 공백으로 대체
    x = x.replace('  ', ' ') # 두 개의 공백을 하나의 공백으로 대체
    x = x.strip() # 앞뒤 공백을 제거
    return x

name_correction(x) : 문자열을 정리하고 표준화하는 작업 수행

Clean item names.

In [ ]:
# 아이템 이름을 첫 번째 괄호로 분할 ex) "item[description]" -> "item", "description]"
items["name1"], items["name2"] = items.item_name.str.split("[", 1).str
items["name1"], items["name3"] = items.item_name.str.split("(", 1).str

# 특수 문자를 제거하고 소문자로 변환 ex) "Description!" -> "description"
items["name2"] = items.name2.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()
items["name3"] = items.name3.str.replace('[^A-Za-z0-9А-Яа-я]+', " ").str.lower()

# 결측값을 '0'으로 채움
items = items.fillna('0')

# name_correction(x)함수를 적용하여 아이템 이름 정리
items["item_name"] = items["item_name"].apply(lambda x: name_correction(x))

# 만약 name2가 '0'이 아니라면, 마지막 문자를 제외한 모든 문자를 반환합니다
# ex) "description]" -> "description"
items.name2 = items.name2.apply( lambda x: x[:-1] if x !="0" else "0")

Clean item type

In [ ]:
items["type"] = items.name2.apply(lambda x: x[0:8] if x.split(" ")[0] == "xbox" else x.split(" ")[0] )
items.loc[(items.type == "x360") | (items.type == "xbox360") | (items.type == "xbox 360") ,"type"] = "xbox 360"
items.loc[ items.type == "", "type"] = "mac"
items.type = items.type.apply( lambda x: x.replace(" ", "") )
items.loc[ (items.type == 'pc' )| (items.type == 'pс') | (items.type == "pc"), "type" ] = "pc"
items.loc[ items.type == 'рs3' , "type"] = "ps3"

'items.name2'열의 각 값에 대해 첫 번째 단어가 'xbox'인 경우, 처음 8글자를 'type'에 저장 , 그렇지 않은 경우 첫 번째 단어를 'type'에 저장
'type'열의 값이 'x360', 'xbox360', 'xbox360'인 행을 찾아 'xbox 360'으로 통일
'type'열의 값이 빈 문자열 -> 'mac'으로 변환
'type'열의 값에서 공백 제거
'type'열의 값이 'pc''pс'"pc" -> "pc"로 통일
'type'열의 값이 'рs3' -> "ps3"으로 통일

In [ ]:
group_sum = items.groupby(["type"]).agg({"item_id": "count"})
group_sum = group_sum.reset_index()
drop_cols = []
for cat in group_sum.type.unique():
    if group_sum.loc[(group_sum.type == cat), "item_id"].values[0] <40:
        drop_cols.append(cat)
items.name2 = items.name2.apply( lambda x: "other" if (x in drop_cols) else x )
items = items.drop(["type"], axis = 1)

* 'type'열 간소화
items 데이터프레임을 'type'열로 그룹화 -> 'item_id' 개수를 계산 -> group_sum에 저장
rest_index() : 그룹화된 결과를 일반 데이터프레임 형태로 변환
drop_cols 빈 리스트 생성
'type'값에 대해 'item_id' 개수가 40개 미만인 경우 해당 'type'값을 drop_cols 리스트에 추가
items.name2 열의 각 값에 대해 'drop_cols'리스트에 포함된 값인 경우 'other'로 변환 
'items' 데이터프레임에서 'type'열을 삭제

In [ ]:
# LabelEncoder를 사용해서 items.name2열의 문자열 값을 숫자형 값으로 변환
items.name2 = LabelEncoder().fit_transform(items.name2) 
items.name3 = LabelEncoder().fit_transform(items.name3)

# items 데이터프레임에서 'item_name'및 'name1'열 삭제 
# (axis=1 : 열 기준으로 삭제하겠다는 뜻, inplace=True : 원래의 데이터프레임 수정한다는 뜻)
items.drop(["item_name", "name1"],axis = 1, inplace= True)
items.head()

# Preprocessing

Create a matrix df with every combination of month, shop and item in order of increasing month. Item_cnt_day is summed into an item_cnt_month.

# 전처리 단계
모든 월, 상점, 아이템의 조합을 포함하는 매트릭스 데이터프레임 생성
아이템의 일별 판매 수량 -> 월별 판매 수량으로 합산

* 매트릭스 데이터프레임이란?
 : 모든 가능한 조합을 포함하는 데이터프레임을 말함
ex. 월 : 1,2  /  상점 : A, B  /  아이템 : X, Y
   -> (1,A,X) (1,A,Y) (1,B,X) (1,B,Y) (2,A,X) (2,A,Y) (2,B,X) (2,B,Y) 

In [ ]:
from itertools import product
import time

# 현재 시간을 ts 변수에 저장하여 실행 시간 측정
ts = time.time() 

# 매트릭스를 생성할 리스트와 열 이름 정의
matrix = []
cols  = ["date_block_num", "shop_id", "item_id"]

# 모든 조합 생성 및 리스트에 추가
for i in range(34):
    sales = train[train.date_block_num == i] 
    matrix.append( np.array(list( product( [i], sales.shop_id.unique(), sales.item_id.unique() ) ), dtype = np.int16) )

# 매트릭스를 데이터프레임으로 변환 (열 이름은 cols 리스트로 지정)
matrix = pd.DataFrame( np.vstack(matrix), columns = cols )

# 데이터프레임의 각 열에 대해 데이터 타입을 지정 : int8
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)

# 데이터 프레임 정렬
matrix.sort_values( cols, inplace = True )

# 현재 시간 - 시작 시간 = 코드 실행 시간 측정 
time.time()- ts

# 모든 조합 생성 및 리스트에 추가
'train' 데이터프레임에서 현재 'date_block_num'값에 해당하는 데이터를 필터링하여 'sales'변수에 저장
'sales' 데이터에서 고유한 'shop_id'와 'item_id'를 추출하고, 'itertools.product'를 사용하여 'date_block_num', 'shop_id', 'item_id'의 모든 가능한 조합을 생성
생성된 조합을 numpy 배열로 변환하여 matrix 리스트에 추가

In [ ]:
# 'train' 데이터프레임에 새로운 열 'revenue' 추가
train["revenue"] = train["item_cnt_day"] * train["item_price"]
#  revenue = item_cnt_day * item_price 

In [ ]:
ts = time.time()

# 'train' 데이터프레임을 "date_block_num", "shop_id", "item_id"로 그룹화 -> item_cnt_day 합계 계산
group = train.groupby( ["date_block_num", "shop_id", "item_id"] ).agg( {"item_cnt_day": ["sum"]} )
group.columns = ["item_cnt_month"] # -> 계산된 결과의 열 이름을 'item_cnt_month'로 변경
group.reset_index( inplace = True) # -> 인덱스 초기화, 그룹화된 결과를 일반 데이터프레임 형태로 변환

# 그룹화된 결과를 matrix 데이터프레임과 병합 
matrix = pd.merge( matrix, group, on = cols, how = "left" )
# on = cols : "date_block_num", "shop_id", "item_id"열 기준으로 
# how='left': matrix 데이터프레임을 기준으로 병합하고, 매칭되지 않는 값은 'NaN'으로 남김

# 'item_cnt_month'열의 결측값 0으로 채우고, 데이터 타입을 'np.float16'으로 변환(메모리 사용량 줄임)
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0).astype(np.float16)

time.time() - ts

Create a test set for month 34.

In [ ]:
# 'date_block_num' 열 추가 , 34로 초기화
test["date_block_num"] = 34

# 데이터타입 np.int8로 변환(메모리 사용량 줄임)
test["date_block_num"] = test["date_block_num"].astype(np.int8)
test["shop_id"] = test.shop_id.astype(np.int8)
test["item_id"] = test.item_id.astype(np.int16)

Concatenate train and test sets.

In [ ]:
ts = time.time()

# 'test' 데이터프레임의 'ID'열 제거 , 'matrix' 데이터프레임에 병합
matrix = pd.concat([matrix, test.drop(["ID"],axis = 1)], ignore_index=True, sort=False, keys=cols)
# ignore_index=True : 병합 후 인덱스 재설정
# sort=False : 열 이름을 기준으로 정렬하지 않음
# keys=cols : 병합된 데이터프레임의 열 이름을 설정

# 결측값 0으로 채움
matrix.fillna( 0, inplace = True )
time.time() - ts

Add shop, items and categories data onto matrix df.

In [ ]:
ts = time.time()

# 'matrix' 데이터프레임에 추가 정보 병합
# 'shops' 데이터프레임을 'matrix' 데이터프레임에 'shop_id'열을 기준으로 병합
# how='left' : matrix 기준으로 병합 , matrix에 없으면 'NaN'으로 남김
matrix = pd.merge( matrix, shops, on = ["shop_id"], how = "left" )
matrix = pd.merge(matrix, items, on = ["item_id"], how = "left")
matrix = pd.merge( matrix, cats, on = ["item_category_id"], how = "left" )

# 데이터 타입 변환(np.int8 or np.int16으로)
matrix["shop_city"] = matrix["shop_city"].astype(np.int8)
matrix["shop_category"] = matrix["shop_category"].astype(np.int8)
matrix["item_category_id"] = matrix["item_category_id"].astype(np.int8)
matrix["subtype_code"] = matrix["subtype_code"].astype(np.int8)
matrix["name2"] = matrix["name2"].astype(np.int8)
matrix["name3"] = matrix["name3"].astype(np.int16)
matrix["type_code"] = matrix["type_code"].astype(np.int8)

time.time() - ts

Feature Engineering

* Add lag features to matrix df.

### 랙 피처(Lag Feature)
 : 시계열 데이터에서 현재 시점의 데이터를 예측하기 위해 과거 시점의 데이터를 사용하는 피처를 말한다. 
   각 랙 피처는 이전 시점의 값을 현재 시점의 피처로 사용
ex. 매달 판매량을 예측하는 모델에서 이전 달이나 이전 몇달의 판매량을 피처로 사용하는 것

### 시계열 데이터(Time Series Data)
 : 시간의 흐름에 따라 순서대로 기록된 데이터
  이러한 데이터는 일정한 시간 간격으로 수집됨. 주로 시간에 따른 변화를 분석하거나 미래를 예측하는 데 사용됨

In [ ]:
def lag_feature( df,lags, cols ):
    for col in cols:
        print(col)  # 'cols' 리스트에 있는 각 열 이름 출력
        
        # "date_block_num", "shop_id","item_id", 그리고 현재 열(col)을 포함하는 임시 데이터프레임 'tmp' 생성
        tmp = df[["date_block_num", "shop_id","item_id",col ]] 
        
        # 랙 피처 생성 및 데이터프레임에 병합
        for i in lags:
            shifted = tmp.copy() # 'tmp' 데이터프레임을 복사
            
            # shifted 열 이름 변경
            shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
            # 'shifted' 데이터프레임의 'date_block_num' 열에 'i'를 더하여 랙을 만듦
            shifted.date_block_num = shifted.date_block_num + i
            
            # 원래의 데이터프레임(df), 'shifted'를 병합 -> 랙 피처를 추가
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df # 랙 피처가 추가된 데이터프레임 반환

lag_feature 함수 매개변수 
df : 랙 피처를 추가할 데이터프레임
lags : 랙의 크기 리스트     ex. [1,2,3] -> 1개월전 2개월전 3개월 전의 값을 사용
cols : 랙 피처를 생성할 열 리스트

'matrix' 데이터프레임에 'item_cnt_month' 열에 대한 1,2,3개월 전 랙 피처 추가

In [ ]:
ts = time.time()
matrix = lag_feature( matrix, [1,2,3], ["item_cnt_month"] )
time.time() - ts

이전 달의 평균 item_cnt_month 추가

In [ ]:
ts = time.time()

# 'date_block_num'별 평균 'item_cnt_month' 계산
group = matrix.groupby( ["date_block_num"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_avg_item_cnt"] # 계산된 평균 값 새로운 열로 저장

# 그룹화된 결과(객체)를 일반 데이터프레임 형식으로 변환, 인덱스 재설정
group.reset_index(inplace = True) 

# matrix , 평균 값 병합
matrix = pd.merge(matrix, group, on = ["date_block_num"], how = "left")
matrix.date_avg_item_cnt = matrix["date_avg_item_cnt"].astype(np.float16) # 데이터 타입 변환

# 'date_avg_item_cnt'열에 대해 1개월 전의 값을 나타내는 랙 피처 추가
matrix = lag_feature( matrix, [1], ["date_avg_item_cnt"] ) 
matrix.drop( ["date_avg_item_cnt"], axis = 1, inplace = True ) # 랙 피처 추가 후 원본 'date_avg_item_cnt'열 삭제

time.time() - ts

월별로 item_id에 대해 item_cnt_month의 랙 값을 추가

In [ ]:
ts = time.time()

# 'date_block_num' 'item_id'로 그룹화 -> 'item_cnt_month' 평균 계산
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ] # 새로운 열로 저장
group.reset_index(inplace=True)

# date_block_num 및 item_id를 기준으로 matrix 데이터프레임과 병합
matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix.date_item_avg_item_cnt = matrix['date_item_avg_item_cnt'].astype(np.float16)
# 'date_item_avg_item_cnt' 열에 대해 1,2,3개월 전의 값을 나타내는 랙 피처 추가
matrix = lag_feature(matrix, [1,2,3], ['date_item_avg_item_cnt'])
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True) # 불필요한 열 삭제

time.time() - ts

월별 및 상점 그룹마다 item_cnt_month의 랙 값을 추가

In [ ]:
ts = time.time()

group = matrix.groupby( ["date_block_num","shop_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num","shop_id"], how = "left")
matrix.date_avg_item_cnt = matrix["date_shop_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1,2,3], ["date_shop_avg_item_cnt"] )
matrix.drop( ["date_shop_avg_item_cnt"], axis = 1, inplace = True )
time.time() - ts

월/상점/아이템 그룹에 대해 item_cnt_month의 랙 값을 추가

In [ ]:
ts = time.time()
group = matrix.groupby( ["date_block_num","shop_id","item_id"] ).agg({"item_cnt_month" : ["mean"]})
group.columns = ["date_shop_item_avg_item_cnt"]
group.reset_index(inplace = True)

matrix = pd.merge(matrix, group, on = ["date_block_num","shop_id","item_id"], how = "left")
matrix.date_avg_item_cnt = matrix["date_shop_item_avg_item_cnt"].astype(np.float16)
matrix = lag_feature( matrix, [1,2,3], ["date_shop_item_avg_item_cnt"] )
matrix.drop( ["date_shop_item_avg_item_cnt"], axis = 1, inplace = True )
time.time() - ts

월/상점/아이템 서브타입 조합에 대해 item_cnt_month의 랙 값을 추가

In [ ]:
ts = time.time()
group = matrix.groupby(['date_block_num', 'shop_id', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_subtype_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'subtype_code'], how='left')
matrix.date_shop_subtype_avg_item_cnt = matrix['date_shop_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_shop_subtype_avg_item_cnt'])
matrix.drop(['date_shop_subtype_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

월/도시 그룹에 대해 item_cnt_month의 랙 값 추가

In [ ]:
ts = time.time()
group = matrix.groupby(['date_block_num', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_city_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', "shop_city"], how='left')
matrix.date_city_avg_item_cnt = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_city_avg_item_cnt'])
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

월/도시/아이템 그룹에 대해 item_cnt_month의 랙 값 추가

In [ ]:
ts = time.time()
group = matrix.groupby(['date_block_num', 'item_id', 'shop_city']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'item_id', 'shop_city'], how='left')
matrix.date_item_city_avg_item_cnt = matrix['date_item_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], ['date_item_city_avg_item_cnt'])
matrix.drop(['date_item_city_avg_item_cnt'], axis=1, inplace=True)
time.time() - ts

* matrix 데이터프레임에 아이템의 평균 가격을 추가
* 각 월별로 아이템 가격의 랙(lag) 값을 추가 
* 현재 월의 평균 가격이 전체 평균 가격과 어떻게 관련되는지에 대한 델타 값(차이 값)을 추가

In [ ]:
ts = time.time()
group = train.groupby( ["item_id"] ).agg({"item_price": ["mean"]})
group.columns = ["item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge( group, on = ["item_id"], how = "left" )
matrix["item_avg_item_price"] = matrix.item_avg_item_price.astype(np.float16)


group = train.groupby( ["date_block_num","item_id"] ).agg( {"item_price": ["mean"]} )
group.columns = ["date_item_avg_item_price"]
group.reset_index(inplace = True)

matrix = matrix.merge(group, on = ["date_block_num","item_id"], how = "left")
matrix["date_item_avg_item_price"] = matrix.date_item_avg_item_price.astype(np.float16)
lags = [1, 2, 3]
matrix = lag_feature( matrix, lags, ["date_item_avg_item_price"] )
for i in lags:
    matrix["delta_price_lag_" + str(i) ] = (matrix["date_item_avg_item_price_lag_" + str(i)]- matrix["item_avg_item_price"] )/ matrix["item_avg_item_price"]

def select_trends(row) :
    for i in lags:
        if row["delta_price_lag_" + str(i)]:
            return row["delta_price_lag_" + str(i)]
    return 0

matrix["delta_price_lag"] = matrix.apply(select_trends, axis = 1)
matrix["delta_price_lag"] = matrix.delta_price_lag.astype( np.float16 )
matrix["delta_price_lag"].fillna( 0 ,inplace = True)

features_to_drop = ["item_avg_item_price", "date_item_avg_item_price"]
for i in lags:
    features_to_drop.append("date_item_avg_item_price_lag_" + str(i) )
    features_to_drop.append("delta_price_lag_" + str(i) )
matrix.drop(features_to_drop, axis = 1, inplace = True)
time.time() - ts

* Add total shop revenue per month to matix df. 
* Add lag values of revenue per month.
* Add delta revenue values - how current month revenue relates to global average.

In [ ]:
ts = time.time()
group = train.groupby( ["date_block_num","shop_id"] ).agg({"revenue": ["sum"] })
group.columns = ["date_shop_revenue"]
group.reset_index(inplace = True)

matrix = matrix.merge( group , on = ["date_block_num", "shop_id"], how = "left" )
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(["shop_id"]).agg({ "date_block_num":["mean"] })
group.columns = ["shop_avg_revenue"]
group.reset_index(inplace = True )

matrix = matrix.merge( group, on = ["shop_id"], how = "left" )
matrix["shop_avg_revenue"] = matrix.shop_avg_revenue.astype(np.float32)
matrix["delta_revenue"] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix["delta_revenue"] = matrix["delta_revenue"]. astype(np.float32)

matrix = lag_feature(matrix, [1], ["delta_revenue"])
matrix["delta_revenue_lag_1"] = matrix["delta_revenue_lag_1"].astype(np.float32)
matrix.drop( ["date_shop_revenue", "shop_avg_revenue", "delta_revenue"] ,axis = 1, inplace = True)
time.time() - ts

Add month and number of days in each month to matrix df.

In [ ]:
matrix["month"] = matrix["date_block_num"] % 12
days = pd.Series([31,28,31,30,31,30,31,31,30,31,30,31])
matrix["days"] = matrix["month"].map(days).astype(np.int8)

Add the month of each shop and item first sale.

In [ ]:
ts = time.time()
matrix["item_shop_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id","shop_id"])["date_block_num"].transform('min')
matrix["item_first_sale"] = matrix["date_block_num"] - matrix.groupby(["item_id"])["date_block_num"].transform('min')
time.time() - ts

Delete first three months from matrix. They don't have lag values.

In [ ]:
ts = time.time()
matrix = matrix[matrix["date_block_num"] > 3]
time.time() - ts

In [ ]:
matrix.head().T

# Modelling

In [ ]:
import gc
import pickle
from xgboost import XGBRegressor
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [ ]:
data = matrix.copy()
del matrix
gc.collect()

In [ ]:
data[data["date_block_num"]==34].shape

Use month 34 as validation for training.

In [ ]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [ ]:
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

In [ ]:
del data
gc.collect();

In [ ]:
ts = time.time()

model = XGBRegressor(
    max_depth=10,
    n_estimators=1000,
    min_child_weight=0.5, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,
#     tree_method='gpu_hist',
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 20)

time.time() - ts

In [ ]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

In [ ]:
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

plot_features(model, (10,14))